In [12]:
import os
import sqlite3
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import warnings

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
from transformers.data.data_collator import DataCollatorWithPadding
from transformers.trainer_callback import EarlyStoppingCallback
from transformers.training_args import TrainingArguments
from transformers.trainer import Trainer

from datasets import Dataset
from tqdm.auto import tqdm

# === Silence warnings/logging ===
warnings.filterwarnings("ignore", message=".*pin_memory.*")
logging.getLogger("transformers").setLevel(logging.ERROR)

# === Model list ===
MODELS = [
    "distilroberta-base",
    "distilbert-base-uncased",
    "nreimers/MiniLM-L6-H384-uncased",
    "google/electra-small-discriminator",
    "huawei-noah/TinyBERT_General_4L_312D"
]

# === Dataset info ===
DATASETS = {
    "news": "data/news.db",
    "reddit": "data/reddit.db",
    "twitter": "data/twitter.db"
}

# === Metric computation ===
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, zero_division=0),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds)
    }

# === Benchmark loop over datasets ===
all_results = []

for source_name, db_path in tqdm(DATASETS.items(), desc="🔬 Benchmarking Datasets"):
    print(f"\n📂 Benchmarking on dataset: {source_name}")
    con = sqlite3.connect(db_path)
    df = pd.read_sql("SELECT summary AS text, label FROM labeled_data", con)
    con.close()
    df.dropna(subset=["text", "label"], inplace=True)   
    df["label"] = df["label"].astype(int)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for model_name in tqdm(MODELS, desc=f"🔬 Benchmarking Models on {source_name}"):
        print(f"\n🔍 Model: {model_name}")
        fold_scores, all_probs, all_labels = [], [], []

        for fold, (train_idx, val_idx) in enumerate(skf.split(df["text"], df["label"])):
            print(f" ↪ Fold {fold+1}/5")
            train_df, val_df = df.iloc[train_idx], df.iloc[val_idx]

            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            safe_model_name = model_name.replace("/", "_").replace("-", "_")
            model_dir = f"./results/{source_name}_{safe_model_name}_fold{fold+1}"

            if os.path.exists(model_dir):
                model = AutoModelForSequenceClassification.from_pretrained(model_dir)
            else:
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

            train_ds = Dataset.from_pandas(train_df).map(lambda x: tokenizer(x["text"], truncation=True, max_length=512), batched=True)
            val_ds = Dataset.from_pandas(val_df).map(lambda x: tokenizer(x["text"], truncation=True, max_length=512), batched=True)

            train_ds = train_ds.rename_column("label", "labels")
            val_ds = val_ds.rename_column("label", "labels")
            train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
            val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

            args = TrainingArguments(
                output_dir=model_dir,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=16,
                num_train_epochs=5,
                eval_strategy="epoch",
                save_strategy="epoch",
                save_total_limit=1,
                load_best_model_at_end=True,
                logging_steps=50,
                report_to="none",
                disable_tqdm=True,
                fp16=False,
                learning_rate=2e-5,
                no_cuda=True,
            )

            checkpoint_path = os.path.join(model_dir, "pytorch_model.bin")
            if os.path.exists(checkpoint_path):
                model = AutoModelForSequenceClassification.from_pretrained(model_dir)
                model.to("cpu")
                train_time = 0.0
            else:
                trainer = Trainer(
                    model=model,
                    args=args,
                    train_dataset=train_ds,
                    eval_dataset=val_ds,
                    compute_metrics=compute_metrics,
                    data_collator=DataCollatorWithPadding(tokenizer),
                    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
                )
                start = time.time()
                trainer.train()
                trainer.save_model(model_dir)
                train_time = round(time.time() - start, 2)

            trainer = Trainer(
                model=model,
                args=args,
                train_dataset=train_ds,
                eval_dataset=val_ds,
                compute_metrics=compute_metrics,
                data_collator=DataCollatorWithPadding(tokenizer),
                callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
            )

            metrics = trainer.evaluate()
            metrics["training_time"] = train_time

            preds = trainer.predict(val_ds)
            probs = torch.nn.functional.softmax(torch.tensor(preds.predictions), dim=1).numpy()
            true_labels = preds.label_ids

            all_probs.append(probs[:, 1])
            all_labels.append(true_labels)

            fpr, tpr, _ = roc_curve(true_labels, probs[:, 1])
            roc_auc = auc(fpr, tpr)
            metrics["auc"] = roc_auc

            os.makedirs("roc_curves", exist_ok=True)
            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"AUC={roc_auc:.2f}")
            plt.plot([0, 1], [0, 1], linestyle='--', color='navy')
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title(f"{model_name} Fold {fold+1} ROC ({source_name})")
            plt.legend(loc="lower right")
            plt.savefig(f"roc_curves/{source_name}_{safe_model_name}_fold{fold+1}.png")
            plt.close()

            fold_scores.append({k.replace("eval_", ""): v for k, v in metrics.items()})

        # Overall ROC curve
        if fold_scores:
            all_probs_flat = np.concatenate(all_probs)
            all_labels_flat = np.concatenate(all_labels)
            fpr, tpr, _ = roc_curve(all_labels_flat, all_probs_flat)
            overall_auc = auc(fpr, tpr)

            plt.figure()
            plt.plot(fpr, tpr, color='green', lw=2, label=f"AUC={overall_auc:.2f}")
            plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title(f"{model_name} Overall ROC ({source_name})")
            plt.legend(loc="lower right")
            os.makedirs("roc_overall", exist_ok=True)
            plt.savefig(f"roc_overall/{source_name}_{safe_model_name}_overall.png")
            plt.close()

            avg = pd.DataFrame(fold_scores).mean().to_dict()
            avg.update({"model": model_name, "auc": overall_auc, "source": source_name})
            all_results.append(avg)

# === Save Final Results ===
results_df = pd.DataFrame(all_results)
required_cols = ["source", "model", "accuracy", "f1", "precision", "recall", "auc", "training_time"]
results_df = results_df[[col for col in required_cols if col in results_df.columns]]
results_df.to_csv("multimodal_results.csv", index=False)
print("\n✅ Multimodal Benchmarking complete. Results saved to multimodal_results.csv")


🔬 Benchmarking Datasets:   0%|          | 0/3 [00:00<?, ?it/s]


📂 Benchmarking on dataset: news


🔬 Benchmarking Models on news:   0%|          | 0/5 [00:00<?, ?it/s]


🔍 Model: distilroberta-base
 ↪ Fold 1/5


Map:   0%|          | 0/340 [00:00<?, ? examples/s]

...

{'loss': 0.6824, 'grad_norm': 1.026371955871582, 'learning_rate': 1.9276752767527677e-05, 'epoch': 0.18450184501845018}
{'loss': 0.6349, 'grad_norm': 3.3838675022125244, 'learning_rate': 1.8538745387453875e-05, 'epoch': 0.36900369003690037}
{'loss': 0.6161, 'grad_norm': 4.0729546546936035, 'learning_rate': 1.7800738007380077e-05, 'epoch': 0.5535055350553506}
{'loss': 0.6133, 'grad_norm': 4.425028324127197, 'learning_rate': 1.7062730627306276e-05, 'epoch': 0.7380073800738007}
{'loss': 0.5743, 'grad_norm': 4.055292129516602, 'learning_rate': 1.6324723247232474e-05, 'epoch': 0.922509225092251}
{'eval_loss': 0.566743791103363, 'eval_accuracy': 0.7134935304990758, 'eval_f1': 0.7832167832167832, 'eval_precision': 0.6896551724137931, 'eval_recall': 0.9061488673139159, 'eval_runtime': 0.6972, 'eval_samples_per_second': 776.0, 'eval_steps_per_second': 48.769, 'epoch': 1.0}
{'loss': 0.5567, 'grad_norm': 8.141851425170898, 'learning_rate': 1.5586715867158672e-05, 'epoch': 1.1070110701107012}
{'lo

Map:   0%|          | 0/2164 [00:00<?, ? examples/s]

Map:   0%|          | 0/541 [00:00<?, ? examples/s]

/Users/gchhetri/Developer/Python/15min-city-bench/.venv/lib/python3.12/site-packages/transformers/training_args.py:1604: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


{'loss': 0.6858, 'grad_norm': 1.826180338859558, 'learning_rate': 1.9276752767527677e-05, 'epoch': 0.18450184501845018}
{'loss': 0.6424, 'grad_norm': 2.0221266746520996, 'learning_rate': 1.8538745387453875e-05, 'epoch': 0.36900369003690037}
{'loss': 0.6435, 'grad_norm': 5.970825672149658, 'learning_rate': 1.7800738007380077e-05, 'epoch': 0.5535055350553506}
{'loss': 0.578, 'grad_norm': 6.533298969268799, 'learning_rate': 1.7062730627306276e-05, 'epoch': 0.7380073800738007}
{'loss': 0.5674, 'grad_norm': 6.677089214324951, 'learning_rate': 1.6324723247232474e-05, 'epoch': 0.922509225092251}
{'eval_loss': 0.5403273105621338, 'eval_accuracy': 0.7486136783733827, 'eval_f1': 0.7951807228915663, 'eval_precision': 0.7436619718309859, 'eval_recall': 0.8543689320388349, 'eval_runtime': 0.6106, 'eval_samples_per_second': 885.966, 'eval_steps_per_second': 55.68, 'epoch': 1.0}
{'loss': 0.5184, 'grad_norm': 10.174674034118652, 'learning_rate': 1.5586715867158672e-05, 'epoch': 1.1070110701107012}
{'l